In [1]:
# 导入配置加载器
from config_loader import load_config

# ==================== 加载配置 ====================
print("=" * 60)
print("加载配置文件")
print("=" * 60)

try:
    config = load_config()
    zhipuai_config = config.get_model_config('zhipuai')
    print(f"\n✅ 配置加载成功！")
    print(f"  模型: {zhipuai_config['model']}")
    print(f"  温度: {zhipuai_config['temperature']}")
except Exception as e:
    print(f"\n❌ 配置加载失败: {e}")
    print("\n请按照以下步骤配置:")
    print("1. 复制 .env.example 为 .env")
    print("   命令: cp .env.example .env")
    print("2. 编辑 .env 文件，填写 ZHIPUAI_API_KEY")
    print("3. 保存后重新运行此单元格")
    raise


加载配置文件
✅ 已加载配置文件: /Users/yuyansong/AiProject/Langchain/langchain-example/langchain/.env

✅ 配置加载成功！
  模型: glm-4.5-air
  温度: 0.7
